# Установка Julia

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots CSV Econometrics DataFrames RDatasets"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.5.2 on the current Colab Runtime...
2021-02-07 06:50:14 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.5/julia-1.5.2-linux-x86_64.tar.gz [105324048/105324048] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed ZMQ ───────────── v1.2.1
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed VersionParsing ── v1.2.0
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.0.15
  Installed IJulia ────────── v1.23.1
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed Conda ─────────── v1.5.0
  Installed JLLWrappers ───── v1.2.0
  Installed SoftGlobalScope ─ v1.1.0
  Installed JSON ──────────── v0.21.1
########################################################################

In [ ]:
versioninfo()

NameError: ignored

In [ ]:
exp(1)

2.718281828459045

# Работу выполнили: 

*   Головизнин Григорий
*   Казакбаев Рустем



# Темы:


*   Форматы данных
*   Строки
*   Функции
*   Массивы данных



# Types

## Некоторые общие слова про форматы данных

У Джулии есть много видов переменных: integers (например 3, -10), float numbers (1.0 и -3.9), bools (Trure / False) и bitarrays (схожи с bool переменными, но они эффективнее используют память компьютера) , строки («привет»), даты (2021-01-25) и многие другие. 

##Integers and Float 

In [ ]:
a = 5                   #integer, Int (Int64 в большинстве случаев)
b = 5.0                 #floating point, (Float64 в большинстве случаев)
A = [1;2]
B = [1.0;2.0]

2-element Array{Float64,1}:
 1.0
 2.0

Проверим командой typeof форматы данных

In [ ]:
println("a: ",typeof(a))
println(a)

a: Int64
5


In [ ]:
println("\nb: ",typeof(b))
println(b)


b: Float64
5.0


In [ ]:
println("\nA: ",typeof(A))
println(A)


A: Array{Int64,1}
[1, 2]


In [ ]:
println("\nB: ",typeof(B))
println(B)


B: Array{Float64,1}
[1.0, 2.0]


## Bools and BitArrays

In [ ]:
c = 5 > 1.1
println("c: ",typeof(c))
println(c)

c: Bool
true


In [ ]:
A = [1;2]

2-element Array{Int64,1}:
 1
 2

In [ ]:
C = A .> 1.5
println("\nC: ",typeof(C))
println(C)


C: BitArray{1}
Bool[0, 1]


In [ ]:
println("BitArray более экономичная версия Bool.\n",
        "Заметьте, что typeof(C[1]) даёт: ",typeof(C[1]))

BitArray более экономичная версия Bool.
Заметьте, что typeof(C[1]) даёт: Bool


Стоит заметить, что, чтобы что-либо применялось к списку не как к одному объекту, а ко всем элементам списка (что верно и для других массивов), необходимо использовать точку после массива.

## Операции с переменными разных форматов и изменение форматов

Выполнение операции суммирования "integer" + "float" работает, и в результате получается число формата float. Подобным образом сумма bool + integer даст integer

In [ ]:
#Сложение integer и float
println("Int + Float64: ",15 + 27.0)

Int + Float64: 42.0


In [ ]:
#Сложение bool и int
println("Bool + Int: ",(10 .> 0) + 1)

Bool + Int: 2


##Перевод формата из Int в Float и наоборот

In [ ]:
A = [2.5 ; 5.3; 100.1]
println("A: ",typeof(A))
println(A)

A: Array{Float64,1}
[2.5, 5.3, 100.1]


Переводим элементы массива из float64 в int

In [ ]:
A_to_Int = round.(Int,A)                     
println("Округление A в Int: ",typeof(A_to_Int))
println(A_to_Int)

Округление A в Int: Array{Int64,1}
[2, 5, 100]


In [ ]:
B = [5;6]
println("B: ",typeof(B))
println(B)

B: Array{Int64,1}
[5, 6]


Изменение формата: Int -> Float64

In [ ]:
B_to_Float64 = convert.(Float64,B)            
println("После перевода B в Float64: ",typeof(B_to_Float64))
println(B_to_Float64)                       #Float64.(A) также работает

После перевода B в Float64: Array{Float64,1}
[5.0, 6.0]


## Перевод из Bool и BitArrays в Int и наоборот 

C = A .> 5

Перевод из BitArray в Int

In [ ]:
A = [3.9 ; 7.3; 20.1]
C = A .> 5

#Перевод из BitArray в Int
C_to_Int = convert.(Int,C)               
println(typeof(C_to_Int))                #Int.(C) также работает
println(C_to_Int)

Array{Int64,1}
[0, 1, 1]


Перевод из int в BitArray

In [ ]:
D = [1;0;1]

D_to_Bool = convert.(Bool,D)             
println(typeof(D_to_Bool))                #Bool.(D) также работает
println(D_to_Bool)

BitArray{1}
Bool[1, 0, 1]


## Проверка типа переменной

Самый простой способ проверить, принадлежит ли переменная определенному типу - это использовать функцию isa(variable, Type).  Тип в функции может быть представлен также в виде массива из нескольких типов. Примеры ниже.

In [ ]:
A = 2.5
B = [3.5,6.5]

2-element Array{Float64,1}:
 3.5
 6.5

Является элемент A числом?

In [ ]:
println("$A is a Number: ",isa(A,Number))

2.5 is a Number: true


Является ли число A integer?

In [ ]:
println("$A is an Int: ",isa(A,Int))

2.5 is an Int: false


Является ли число A integer или float?

In [ ]:
println("$A is an Int or a Float64: ",isa(A,Union{Int,Float64}))

2.5 is an Int or a Float64: true


Является ли элемент B числом формата float?

In [ ]:
println("$B is a Float64: ",isa(B,Float64))

[3.5, 6.5] is a Float64: false


Является ли B массивом?

In [ ]:
println("$B is an Array: ",isa(B,Array))

[3.5, 6.5] is an Array: true


# Строки

## Базовые операции


Далее будет показано, как:

*   соединять несколько строк в одну строку, используя string(str1, str2) или str1 * str2
*   проверять, содержит ли строка конкретную подстроку
*   заменять часть строки чем-либо
*   разделять строку в массив слов (и затем объединять их назад в строку)
*   сортировать вектор слов в алфавитном порядке


In [ ]:
str1 = "Привет"
str2 = "Мир!\n"
str3 = "Как твои дела?"

new_str = string(str1," ",str2,str3)          #объединить в одну строку
println(new_str)

Привет Мир!
Как твои дела?


Проверяем, содержит ли строка "Я посетил США" слово "США".

In [ ]:
str4 = "Я посетил США"

if occursin("США",str4)
    println("Предложение")
    println(str4)
    println("содержит слово 'США'")
end

Предложение
Я посетил США
содержит слово 'США'


Заменяем слово "США" на "Германию"

In [ ]:
str4 = replace(str4,"США" => "Германию")
println("\nСтрока после замены: ")
println(str4)


Строка после замены: 
Я посетил Германию


Разделим предложение на слова, представленные в виде массива

In [ ]:
words = split(str4)
println("Разделение предложения в массив слов:")
println(words)

Разделение предложения в массив слов:
SubString{String}["Я", "посетил", "Германию"]


А теперь снова соединим слова в предложение

In [ ]:
println("\nА теперь объединение слов назад в строку:")
println(join(words," "))


А теперь объединение слов назад в строку:
Я посетил Германию


Отсортируем слова в алфавитном порядке

In [ ]:
println("Отсортировать в алфавитном порядке:")
println(sort(words,lt=isless))

Отсортировать в алфавитном порядке:
SubString{String}["Германию", "Я", "посетил"]


##Сроки и индексы

Чтобы обратиться к  i-ому элементу строки str можно использовать str[i]

In [ ]:
str = "Привет, мир!"
println(str)
println(string("Второй элемент строки: ", 'р'))

Привет, мир!
Второй элемент строки: р


Стоит отметить, что может возникнуть ситуация, когда элемент в строке не будет выводиться данной командой, поскольку он требует больше 1 бита для хранения, и будет выдаваться ошибка. В таком случае есть обход в виде следующей команжды:

In [ ]:
str = "Δx = -0.9x"
println(str[nextind(str,1)])

x


## Проход по всем символам в строке

In [ ]:

i = 1
for j in str               
    #global i              
    println(i," ",j)
    i = nextind(str,i)     
end

1 Δ
3 x
4  
5 =
6  
7 -
8 0
9 .
10 9
11 x


## Больше общих операций

Можно лексикографически сравнивать строки, используя стандартные операторы сравнения:

In [ ]:
"abracadabra" < "xylophone"

true

In [ ]:
"abracadabra" == "xylophone"

false

In [ ]:
"Hello, world." != "Goodbye, world."

true

In [ ]:
"1 + 2 = 3" == "1 + 2 = $(1 + 2)"

true

Можно искать индекс конкретного символа, используя функции findfirst и findlast:

In [ ]:
findfirst(isequal('а'), "машина")

3

In [ ]:
findlast(isequal('и'), "велосипед")

11

In [ ]:
findfirst(isequal('т'), "самолет")

13

Можно начать поиск символа по заданному смещению, используя функции findnext и findprev:

In [ ]:
findnext(isequal('o'), "xylophone", 1)

4

In [ ]:
findnext(isequal('o'), "xylophone", 5)

7

In [ ]:
findprev(isequal('o'), "xylophone", 5)

4

In [ ]:
findnext(isequal('o'), "xylophone", 8)

# Функции

## Функции с одним выводом

В коде часто необходимо выполнять один и тот же набор команд, например мне необходим поиск дискриминанта. Для удобной реализации данных команд лучше всего использовать функции. Базовый подход к написанию функции выглядит следующим образом.



```
function yourFunction(a, b, c)
      ... ваш код
      return D
end
```

После того как я определил функцию, я могу вызывать ее в любом удобном для меня месте. Например, 

```
someVariable = yourFunction(1, 2, 4)
```

В переменной `someVariable` будет храниться значение, полученное в функции `yourFunction` с значениями коэффицентов $1, 2, 4$.

Перейду непосредственно к написании функции `Discriminant`, которая на вход берет значения коэффиентов квадратного уравнения $a, b, c$ ($ax^2+bx+c=0$), а на выходе возвращает дискриминант.



In [ ]:
function Discriminant(a, b, c)
      D = b^2 - 4 * a * c
      return D
end

Discriminant (generic function with 1 method)

Рассмотрим простое уравнение $x^2 + 4x + 3$. Дискриминант в нем равен $D = b^2 - 4ac = 4^2 - 4 \cdot 1 \cdot 3 = 16 - 12 = 4$. Теперь вызовем написанную функцию с параметрами (1, 4, 3)

In [ ]:
Discriminant(1, 4, 3)

4

Получаем заветный ответ 4. Прелесть функций что я могу подставить в нее любые значения коэффицентов, и мне не нужно для этого опять писать один и тот же код. Посмотрим теперь на уравнение $x^2+7x+6=0$

In [ ]:
Discriminant(1, 7, 6)

25

### Дефолтные значения для функций

В Julia можно создавать дефолтные значения для коэффициентов. Представим, что я хочу построить длину доверительного интервала для коэффициентов регрессии. Эмпирически, я знаю что почти всегда мое $t_{crit}$ это по модулю 2 ($t_{crit} = |2|$). Соотвественно, я могу написать функцию, задав необходимое мне значение $t_{crit}$.

Замечу, что дефолтные значения должны идти в самом конце входных переменных.

In [ ]:
function ciLength(se_bhat, t_crit = 2)
      ciLength = 2 * se_bhat * t_crit
      return ciLength
end

ciLength (generic function with 2 methods)

Допустим, что $s.e.(\hat{b}_0) = 0.5$. Посчитаем длину доверительного интервала

In [ ]:
ciLength(0.5)

2.0

Отмечу, что мне не пришлось прописывать дефолтное значение $t_{crit}$. Теперь я могу вызывать данную функцию в любой части моего кода и находить длину доверительного интервала.

### Функции поэлементно

Не всегда бывает ситуация, когда мои критические значения равны $|2|$. Существует случаи, что они равны например, $1.64$, что существенно повлияет на ответ. 

Теперь я хочу вызывать функцию, перебирая мои возможные искомые значения критических значений: $[1.64, 1.96, 2]$ для следующих стандратных ошибок $[0.5, 0.8, 1.5]$. Соотвественно, стандартной ошибке $0.5$ соотвествует критическое значение $1.64$.

На выходе у меня должно получится три значения. Базовый подход для осуществления данной идее выглядит следующим образом. Достаточно, задать списки и подставить точку `.` после функции.

```
firstArray = [1, 2, 3]
secondArray = [4, 5, 6]
yourFunction.(firstArray, secondArray)
```



In [ ]:
seArray = [0.5, 0.8, 1.5]
tcritArray = [1.64, 1.96, 2]

ciLength.(seArray, tcritArray)

3-element Array{Float64,1}:
 1.64
 3.136
 6.0

Как и ожидалось я получил 3 значения. Для первого случая оно равно 1.64, по сути происходит следующая команда для случая когда индекс равен единице.

In [ ]:
ciLength(seArray[1], tcritArray[1])

1.64

## Функции с множественным выводом

### Базовый подход для функций с множественным выводом

Конечно, функции могут возвращать не только одно значения, а несколько значений. Вернемся к примеру с дискриминантом, теперь мы хотим не только возвращать значения дискримината, но и его корни, реализую это следующим образом.

In [ ]:
function solveQuadratic(a, b, c)
    D = b^2 - 4 * a * c
    x1 = (-b - D^0.5) / 2 * a
    x2 = (-b + D^0.5) / 2 * a
    return D, x1, x2
end

solveQuadratic (generic function with 1 method)

Теперь, возьмем квадратное выражение, например $x^2 - 8x + 12 = 0$. По теореме Виета очевидно, что корни будут следующими $x_1 = 2, x_2 = 6$. Воспользуемся функцией, чтобы решить данное уравнение.

In [ ]:
solveQuadratic(1, -8, 12)

(16, 2.0, 6.0)

Получаем искомое значение в формате кортежа. 

Допустим, мне нужны только корни уравнения и я не хочу никак видеть дискриминант - он меня сбивает и из-за невнимательности я могу его написать в контрольной. Реализовать это можно следующим образом.



```
(_, Output1, Output2) = yourFunction(a, b, c)
```



In [ ]:
(_, x1, x2) = solveQuadratic(1, -8, 12)
println("First solution: ", x1, " | Second solution: ", x2)

First solution: 2.0 | Second solution: 6.0


А теперь я хочу получать только дискриминант, без корней выражения. Реализуется это следующим образом

In [ ]:
(D,) = solveQuadratic(1, -8, 12)
println("Discriminant: ", D)

Discriminant: 16


Каждый раз писать в print `First Solution`, `Second solution`, `Discriminant` и так далее немного сложно. К тому же, это может привести к внезапным ошибкам в коде. Лучше задать имена возвращенным значения в функции.



In [ ]:
function solveQuadraticWithNames(a, b, c)
    D = b^2 - 4 * a * c
    x1 = (-b - D^0.5) / 2 * a
    x2 = (-b + D^0.5) / 2 * a
    solveTuple = (Discriminant = D, x_1 = x1, x_2 = x2)
    return solveTuple
end

solveQuadraticWithNames (generic function with 1 method)

In [ ]:
solveQuadraticWithNames(1, -8, 12)

(Discriminant = 16, x_1 = 2.0, x_2 = 6.0)

Теперь возвращаются подписанные значения, соотвественно риск что-то напутать в принте минимален и код становится более читаемым.

Раньше я писал о том, что можно перебирать список для функции. Попробуем реализовать тоже самое в функции с множественным выводом.

In [ ]:
solveQuadraticWithNames.([5, 1, 2], 10, 5)

3-element Array{NamedTuple{(:Discriminant, :x_1, :x_2),Tuple{Int64,Float64,Float64}},1}:
 (Discriminant = 0, x_1 = -25.0, x_2 = -25.0)
 (Discriminant = 80, x_1 = -9.47213595499958, x_2 = -0.5278640450004204)
 (Discriminant = 60, x_1 = -17.745966692414832, x_2 = -2.254033307585166)

Конечно,  все работает и для функции с множественным выводом. В данном случае получаем корни уравнений 
$(1) 5x^2 + 10x + 5 = 0; (2) x^2 + 10x + 5 = 0;  (3) 2x^2+10x+5=0$

Пожалуй, это одни из наиболее важных деталей, которые касаются функций в Julia. Далее я перейду к теме Data Containers.

# Data Containers

## Списки
Один из наиболее популярных видов структуры данных.

Базовый подход к созданию списков через квадратные скобки. Допустим, у меня есть данные о объеме сделок по акциям Сбербанка на московской бирже за 5 дней. Тогда мой список будет выглядить следующим образом.

In [ ]:
sberbankArray = [1 2 1 3 4]

1×5 Array{Int64,2}:
 1  2  1  3  4

Если я хочу узнать объем торгов сбербанком за i-ый день могу воспользоваться следующей командой.


```
sberbankArray[i]
```



Узнаю объем торгов Сбером за первый день

In [ ]:
sberbankArray[1]

1

Крокодил Утундрий разведал данные и теперь у меня есть данные и об объемах сделок акциями Facebook и Google за 5 дней. У Facebook $[4, 3, 2, 1, 5]$ миллионов сделок, у Google $[1, 5, 3, 7, 2]$

In [ ]:
sellyourdataArray = [4 3 2 1 5;1 5 7 3 2]

2×5 Array{Int64,2}:
 4  3  2  1  5
 1  5  7  3  2

Теперь я могу объединить эти данные для этого создам новый список из двух выше

In [ ]:
stocksArray = [sberbankArray; sellyourdataArray]

3×5 Array{Int64,2}:
 1  2  1  3  4
 4  3  2  1  5
 1  5  7  3  2

Теперь у меня есть список размера 3x5, в котором хранятся объемы сделок по акциям :)

## Кортежи

Смысл кортежей в том, что они неизменяемые. Кортежи используются как входные и выходные данные в функциях - мы кстати их и использовали, когда работали с функциями

In [ ]:
randomArray = rand(1, 3) ### Сгенерируем рандомно список из трех элементов
loveJulia = "I love julia" ### Создадим string (на русском строковой тип)

randomTuple = (randomArray, loveJulia) ### Создаем кортеж или tuple

([0.05525510499842401 0.9709768648042698 0.7928867570136211], "I love julia")

В кортеже выше смысла не так много, однако иногда необходимы осознанные кортежи с названиями. Сейчас я создам кортеж, где будет название дискриминанту и корням квадратного уравнения.


In [ ]:
solveTuple = (Discriminant = 10, x_1 = 1, x_2 = 2)

(Discriminant = 10, x_1 = 1, x_2 = 2)

Такую же логику мы использовали, когда писали функцию для нахождения корней квадратного уравнения.

Конечно, можно найти дискриминант, указав его индекс в кортеже

In [ ]:
solveTuple[1]

10

Дискриминант можно найти и иначе, указав его название следующим образом:

In [ ]:
solveTuple.Discriminant

10

Или можно достать string или список из первого кортежа

In [ ]:
randomTuple[1]

1×3 Array{Float64,2}:
 0.0552551  0.970977  0.792887

Или loveJulia

In [ ]:
randomTuple[2]

"I love julia"

Таким образом, мы научились создавать кортежи и вставлять туда списки, данные строчного вида(strings), integers и не только. Более того, мы научились называть данные в кортеже и обращаться к ним.

## Словарь

У меня есть название компании на бирже и их тикеры(tickers), например компания Apple называется как ни странно APPLE, однако их тикер на бирже "AAPL", у Facebook: "FB".

Необходимо эти данные структировать в одном месте, для этого используется словари. Словари в Julia создаются по следующему принципу: 



```
yourDict = Dict("key"=>"value")
```



In [ ]:
stockDict = Dict("APPL"=>"Apple", "FB"=>"Facebook")  

Dict{String,String} with 2 entries:
  "FB"   => "Facebook"
  "APPL" => "Apple"

Предположим, что я хочу узнать какой компании принадлежит тикер "FB"

In [ ]:
stockDict["FB"]

"Facebook"

Теперь знаю, что принадлежит Фейсбуку!

Крокодил утундрий узнал новую для нас компанию: Google с тикером "GOOGL". Добавим ее в наш словарь.

In [ ]:
stockDict[:"GOOGL"] = "Google"

"Google"

Теперь посмотрим на наш словарь

In [ ]:
display(stockDict)

Dict{String,String} with 3 entries:
  "GOOGL" => "Google"
  "FB"    => "Facebook"
  "APPL"  => "Apple"

Хочу более читаемые данные, тогда воспользуемся циклом и покажем ему что в этих данных есть что.

In [ ]:
for (key,value) in stockDict                #loop over a dictionary
    println("Ticker: ", "$key | ", "Company name: ", "$value")
end

Ticker: GOOGL | Company name: Google
Ticker: FB | Company name: Facebook
Ticker: APPL | Company name: Apple


## Собственный контейнер

Иногда полезно создать собственный контейнер из данных.

Мой тип данных будет состоять из цифр вначале, далее string и далее список. 

In [ ]:
dogAge = 3 
dogName = "Chop"
dogGirlfriendsArray = ["Sara", "Ri", "Eric"]

3-element Array{String,1}:
 "Sara"
 "Ri"
 "Eric"

Создаю данные через struct

In [ ]:
struct dog        
   dogAge::Int        # Возраст должен быть цифрой
   dogName::String    # Имя должно быть string
   dogGirlfriendsArray::Array # Список из собак подружек    
end

In [ ]:
Chop = dog(dogAge, dogName, dogGirlfriendsArray)

dog(3, "Chop", ["Sara", "Ri", "Eric"])

Могу легко узнать возраст собаки по кличке Chop

In [ ]:
Chop.dogAge

3

Узнать его полное имя

In [ ]:
Chop.dogName

"Chop"

и посмотреть его подружек

In [ ]:
Chop.dogGirlfriendsArray

3-element Array{String,1}:
 "Sara"
 "Ri"
 "Eric"

А в финале посмотреть полную карточку собаки Chop

In [ ]:
Chop

dog(3, "Chop", ["Sara", "Ri", "Eric"])

Таким образом, создание собственных структур данных часто бывает удобно. Прикладной пример - создание карточки собаки для ветеринарной клиники.